In [29]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [30]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
os.environ['OPENAI_API_KEY']=os.getenv("OPENAI_API_KEY")
GROQ_API_KEY = os.getenv("GROQ_API_KEY")

pdf_files = ["2022-sustainability-report.pdf"]

docs = []
for pdf in pdf_files:
    loader = PyMuPDFLoader(pdf)
    loaded_docs = loader.load()
    for doc in loaded_docs:
        doc.metadata['source'] = pdf
        doc.metadata['page'] = doc.metadata.get('page', 1)  
    docs.extend(loaded_docs)

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=250, chunk_overlap=0
)
doc_splits = text_splitter.split_documents(docs)

vectorstore = Chroma.from_documents(
    documents=doc_splits,
    collection_name="rag-chroma",
    embedding=OpenAIEmbeddings(),
)
retriever = vectorstore.as_retriever()

In [31]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_groq import ChatGroq



class GradeDocuments(BaseModel):

    binary_score: str = Field(
        description="Documents are relevant to the question, 'yes' or 'no'"
    )

llm=ChatGroq(groq_api_key=GROQ_API_KEY,
             model_name="Llama3-8b-8192", temperature=0)

structured_llm_grader = llm.with_structured_output(GradeDocuments)

system = """You are a grader assessing relevance of a retrieved document to a user question. \n 
    It does not need to be a stringent test. The goal is to filter out erroneous retrievals. \n
    If the document contains keyword(s) or semantic meaning related to the user question, grade it as relevant. \n
    Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the question."""
grade_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "Retrieved document: \n\n {document} \n\n User question: {question}"),
    ]
)

retrieval_grader = grade_prompt | structured_llm_grader
question = "agent memory"
docs = retriever.get_relevant_documents(question)
doc_txt = docs[1].page_content
print(retrieval_grader.invoke({"question": question, "document": doc_txt}))

binary_score='no'


In [32]:
### Generate

from langchain import hub
from langchain_core.output_parsers import StrOutputParser

def format_docs(docs):
    formatted_docs = []
    for doc in docs:
        formatted_doc = f"{doc.page_content}\n(Source: {doc.metadata['source']}, Page: {doc.metadata['page']})"
        formatted_docs.append(formatted_doc)
    return "\n\n".join(formatted_docs)

rag_prompt = hub.pull("rlm/rag-prompt-llama")
citation_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. When answering the question, use the provided context and include citations in the format (Source: [document name], Page: [page number])."),
    ("human", "{question}"),
    ("human", "Context: {context}")
])

rag_chain = citation_prompt | llm | StrOutputParser()

generation = rag_chain.invoke({"context": docs, "question": question})
print(generation)

Based on the provided context, it appears that Amazon's agent memory plays a crucial role in reducing veteran suicide by coordinating with the U.S. Department of Veterans Affairs and other organizations. The company's resources are leveraged to support veteran service organizations globally, and initiatives like Program Honor are established to advance suicide prevention efforts.

According to the report, Amazon's agent memory is used to:

* Assist in reducing veteran suicide by coordinating with the U.S. Department of Veterans Affairs and other organizations.
* Provide support to finalists through an accelerator course and technical mentorship.
* Grow partnerships with organizations like the Royal British Legion and advance initiatives like Program Honor.

(Source: Building a Better Future Together 2022 Amazon Sustainability Report, Page: 64)

It is essential to note that the exact details of Amazon's agent memory and its role in reducing veteran suicide are not explicitly stated in t

In [33]:
### Hallucination Grader

class GradeHallucinations(BaseModel):
    """Binary score for hallucination present in generation answer."""

    binary_score: str = Field(
        description="Answer is grounded in the facts, 'yes' or 'no'"
    )

structured_llm_grader = llm.with_structured_output(GradeHallucinations)

system = """You are a grader assessing whether an LLM generation is grounded in / supported by a set of retrieved facts. \n 
     Give a binary score 'yes' or 'no'. 'Yes' means that the answer is grounded in / supported by the set of facts."""
hallucination_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "Set of facts: \n\n {documents} \n\n LLM generation: {generation}"),
    ]
)

hallucination_grader = hallucination_prompt | structured_llm_grader
hallucination_grader.invoke({"documents": docs, "generation": generation})

GradeHallucinations(binary_score='yes')

In [34]:
### Answer Grader

class GradeAnswer(BaseModel):
    """Binary score to assess answer addresses question."""

    binary_score: str = Field(
        description="Answer addresses the question, 'yes' or 'no'"
    )

structured_llm_grader = llm.with_structured_output(GradeAnswer)

system = """You are a grader assessing whether an answer addresses / resolves a question \n 
     Give a binary score 'yes' or 'no'. Yes' means that the answer resolves the question."""
answer_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "User question: \n\n {question} \n\n LLM generation: {generation}"),
    ]
)

answer_grader = answer_prompt | structured_llm_grader
answer_grader.invoke({"question": question, "generation": generation})

GradeAnswer(binary_score='no')

In [35]:
### Question Re-writer

system = """You a question re-writer that converts an input question to a better version that is optimized \n 
     for vectorstore retrieval. Look at the input and try to reason about the underlying semantic intent / meaning."""
re_write_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        (
            "human",
            "Here is the initial question: \n\n {question} \n Formulate an improved question.",
        ),
    ]
)

question_rewriter = re_write_prompt | llm | StrOutputParser()
question_rewriter.invoke({"question": question})

'Here\'s a rewritten version of the question that\'s optimized for vectorstore retrieval:\n\n"What are the key concepts related to an agent\'s cognitive abilities, particularly in terms of information storage and retrieval?"\n\nThis rewritten question aims to capture the underlying semantic intent of the original question, which appears to be about an agent\'s ability to store and retrieve information. By using more specific and technical terms like "cognitive abilities", "information storage", and "retrieval", the question becomes more precise and targeted, making it easier for a vectorstore to retrieve relevant documents and concepts.\n\nAdditionally, the rewritten question uses a more formal and technical tone, which is more suitable for a vectorstore retrieval system. This can help to improve the accuracy and relevance of the search results.'

In [36]:
from typing import List

from typing_extensions import TypedDict


class GraphState(TypedDict):
    """
    Represents the state of our graph.

    Attributes:
        question: question
        generation: LLM generation
        documents: list of documents
    """

    question: str
    generation: str
    documents: List[str]
    citations: List[str]

In [37]:
### Nodes

def retrieve(state):

    print("---RETRIEVE---")
    question = state["question"]

    # Retrieval
    documents = retriever.get_relevant_documents(question)
    return {"documents": documents, "question": question}


def generate(state):
    print("---GENERATE---")
    question = state["question"]
    documents = state["documents"]

    context = format_docs(documents)
    generation = rag_chain.invoke({"context": context, "question": question})
    
    citations = []
    for doc in documents:
        if f"(Source: {doc.metadata['source']}, Page: {doc.metadata['page']})" in generation:
            citations.append({
                "source": doc.metadata['source'],
                "page": doc.metadata['page']+1
            })
    
    return {"documents": documents, "question": question, "generation": generation, "citations": citations}


def grade_documents(state):
    print("---CHECK DOCUMENT RELEVANCE TO QUESTION---")
    question = state["question"]
    documents = state["documents"]

    filtered_docs = []
    for d in documents:
        score = retrieval_grader.invoke(
            {"question": question, "document": d.page_content}
        )
        grade = score.binary_score
        if grade == "yes":
            print("---GRADE: DOCUMENT RELEVANT---")
            filtered_docs.append(d)
        else:
            print("---GRADE: DOCUMENT NOT RELEVANT---")
            continue
    return {"documents": filtered_docs, "question": question}


def transform_query(state):
    print("---TRANSFORM QUERY---")
    question = state["question"]
    documents = state["documents"]

    better_question = question_rewriter.invoke({"question": question})
    return {"documents": documents, "question": better_question}


### Edges


def decide_to_generate(state):
    print("---ASSESS GRADED DOCUMENTS---")
    state["question"]
    filtered_documents = state["documents"]
    if not filtered_documents:
            print("---DECISION: ALL DOCUMENTS ARE NOT RELEVANT TO QUESTION, TRANSFORM QUERY---")
            return "transform_query"
    else:
        print("---DECISION: GENERATE---")
        return "generate"


def grade_generation_v_documents_and_question(state):
    print("---CHECK HALLUCINATIONS---")
    question = state["question"]
    documents = state["documents"]
    generation = state["generation"]

    score = hallucination_grader.invoke(
        {"documents": documents, "generation": generation}
    )
    grade = score.binary_score

    if grade == "yes":
        print("---DECISION: GENERATION IS GROUNDED IN DOCUMENTS---")
        print("---GRADE GENERATION vs QUESTION---")
        score = answer_grader.invoke({"question": question, "generation": generation})
        grade = score.binary_score
        if grade == "yes":
            print("---DECISION: GENERATION ADDRESSES QUESTION---")
            return "useful"
        else:
            print("---DECISION: GENERATION DOES NOT ADDRESS QUESTION---")
            return "not useful"
    else:
        pprint("---DECISION: GENERATION IS NOT GROUNDED IN DOCUMENTS, RE-TRY---")
        return "not supported"

In [38]:
from langgraph.graph import END, StateGraph

workflow = StateGraph(GraphState)

workflow.add_node("retrieve", retrieve)  # retrieve
workflow.add_node("grade_documents", grade_documents)  # grade documents
workflow.add_node("generate", generate)  # generatae
workflow.add_node("transform_query", transform_query)  # transform_query

workflow.set_entry_point("retrieve")
workflow.add_edge("retrieve", "grade_documents")
workflow.add_conditional_edges(
    "grade_documents",
    decide_to_generate,
    {
        "transform_query": "transform_query",
        "generate": "generate",
    },
)
workflow.add_edge("transform_query", "retrieve")
workflow.add_conditional_edges(
    "generate",
    grade_generation_v_documents_and_question,
    {
        "not supported": "generate",
        "useful": END,
        "not useful": "transform_query",
    },
)
app = workflow.compile()

In [39]:
from pprint import pprint

inputs = {"question": "Tell me about Prioritizing Recyclability and Carbon Impact?"}
for output in app.stream(inputs):
    for key, value in output.items():
        pprint(f"Finished running '{key}':")
    pprint("\n---\n")

print("Generation:")
print(value["generation"])
print("\nCitations:")
for citation in value["citations"]:
    print(f"- {citation['source']}, Page: {citation['page']}")

---RETRIEVE---
"Finished running 'retrieve':"
'\n---\n'
---CHECK DOCUMENT RELEVANCE TO QUESTION---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---ASSESS GRADED DOCUMENTS---
---DECISION: GENERATE---
"Finished running 'grade_documents':"
'\n---\n'
---GENERATE---
---CHECK HALLUCINATIONS---
---DECISION: GENERATION IS GROUNDED IN DOCUMENTS---
---GRADE GENERATION vs QUESTION---
---DECISION: GENERATION ADDRESSES QUESTION---
"Finished running 'generate':"
'\n---\n'
Generation:
Based on the provided context, it appears that Amazon is prioritizing recyclability and carbon impact in their packaging and waste management strategies. Here are some key points:

1. Recyclable paper-based packaging: Amazon's recyclable paper-based packaging helps customers divert waste from landfills (Source: 2022-sustainability-report.pdf, Page: 32).
2. Reducing packaging waste: Amazon is working to reduce packaging waste by sending more m

In [40]:
inputs = {"question": "What are Amazon's goals regarding Waste and Circularity?"}
for output in app.stream(inputs):
    for key, value in output.items():
        pprint(f"Finished running '{key}':")
    pprint("\n---\n")

# Final generation
print("Generation:")
print(value["generation"])
print("\nCitations:")
for citation in value["citations"]:
    print(f"- {citation['source']}, Page: {citation['page']}")

---RETRIEVE---
"Finished running 'retrieve':"
'\n---\n'
---CHECK DOCUMENT RELEVANCE TO QUESTION---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---ASSESS GRADED DOCUMENTS---
---DECISION: GENERATE---
"Finished running 'grade_documents':"
'\n---\n'
---GENERATE---
---CHECK HALLUCINATIONS---
---DECISION: GENERATION IS GROUNDED IN DOCUMENTS---
---GRADE GENERATION vs QUESTION---
---DECISION: GENERATION ADDRESSES QUESTION---
"Finished running 'generate':"
'\n---\n'
Generation:
According to Amazon's 2022 Sustainability Report, the company's goals regarding waste and circularity include:

1. Eliminating waste across Amazon's own operations, as well as throughout its supply chain and customer areas. (Source: 2022-sustainability-report.pdf, Page: 33)
2. Scaling waste reduction and product circularity programs across Amazon's own operations, as well as throughout its supply chain and customer areas. (Source: 2022-sustai

In [41]:
inputs = {"question": "How is Amazon helping in upskilling their employees?"}
for output in app.stream(inputs):
    for key, value in output.items():
        pprint(f"Finished running '{key}':")
    pprint("\n---\n")

print("Generation:")
print(value["generation"])
print("\nCitations:")
for citation in value["citations"]:
    print(f"- {citation['source']}, Page: {citation['page']}")

---RETRIEVE---
"Finished running 'retrieve':"
'\n---\n'
---CHECK DOCUMENT RELEVANCE TO QUESTION---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---ASSESS GRADED DOCUMENTS---
---DECISION: GENERATE---
"Finished running 'grade_documents':"
'\n---\n'
---GENERATE---
---CHECK HALLUCINATIONS---
---DECISION: GENERATION IS GROUNDED IN DOCUMENTS---
---GRADE GENERATION vs QUESTION---
---DECISION: GENERATION ADDRESSES QUESTION---
"Finished running 'generate':"
'\n---\n'
Generation:
Amazon is helping its employees upskill through various initiatives. According to the 2022 Sustainability Report, Amazon invested $1 billion to upskill over 100,000 U.S. employees. Additionally, the company provides free skills training to over 10 million people globally, aiming to equip them with the skills needed to succeed in the modern workforce.

Amazon's Future Engineer program is another example of the company's efforts to upskill its 